# VCノードの起動

---

CoursewareHubの構築環境となるVCノードを起動して、そのうえにDocker Swarmの設定を行います。

## 構成

CoursewareHubを構成するコンテナ(`auth-proxy`, `JupyterHub`,...)はVCノードに設定するDocker Swarmのうえで実行します。

![構成](images/cw-001-01.png)

# VCノードに関するパラメータ

CoursewareHubの構築環境となるVCノードに関するパラメータを指定します。

この章では、まずVCPの操作を行うために必要となるアクセストークンの値を入力します。次に構築環境のホストとなるVCノードに関するパラメータの指定を行います。最後に入力したパラメータを他のNotebookから参照できるようにNotebook環境のファイルに保存します。

## VCCアクセストークンの入力

VCノードを起動するにはVC Controller(VCC)にアクセスして、操作を行う必要があります。VCCにアクセスするために必要となるアクセストークンをここで入力します。

次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
2018-09-XX XX:XX:XX,XXX - ERROR - config vc failed: http_status(403)
2018-09-XX XX:XX:XX,XXX - ERROR - 2018/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

## クラウドプロバイダの指定

VCノードを起動するプロバイダを選択します。

In [ ]:
# (例)
# vc_provider = 'aws'
# vc_provider = 'azure'

vc_provider = 'aws'

## VCノードに割り当てるリソース量の指定


VCノードに割り当てるリソース量を`spec`オブジェクトに指定します。

`spec`オブジェクトの全てのパラメータを毎回設定するのは煩雑になるので、VCP SDKでは典型的な構成のパラメータセットを事前に定義しています。事前に定義した`spec`パラメータセットのことを`flavor`と呼んでいます。`spec`に設定できるパラメータはクラウドプロバイダ毎に異なるので `flavor`もプロバイダ毎にの定義されています。

次のセルを実行すると `vc_provider` に設定したプロバイダに対応する `flavor` の一覧が表示されます。

In [ ]:
vcp.df_flavors(vc_provider)

表示された `flavor` の値から一つを選択して、次のセルに指定してください。

In [ ]:
# (例)
# vc_flavor = 'small'

vc_flavor =

CoursewareHub環境ではコンテナイメージとして、4GB以上のサイズのものを利用します。
`small`の `flavor`に対応するルートボリュームのサイズ８GBでは、コンテナを起動するのにディスク容量が不足してしまいます。そこでルートボリュームサイズを`flavor`に定義されている値から変更します。

次のセルでルートボリュームサイズ(GB)を指定してください。

In [ ]:
# (例)
# vc_disk_size = 16

# 16GB以上の値を指定してください
vc_disk_size =

## ノード数の指定

起動するVCノード数を指定してください。指定されたノード数のうち１ノードを manager 用、残りのノード数を worker 用のVCノードとして利用します。

> ノード数を１に設定すると manager ノードのみで構成されます。

In [ ]:
# (例)
# vc_nodes = 3

vc_nodes =

## プライベートIPアドレスの指定

CoursewareHubを実行するVCノードは利用者からアクセスされるので、事前に定めたIPアドレスを割り当てたほうが管理しやすくなります。
ここでは、割りあてるIPアドレスの指定を行います。

次のセルでVCノードに割り当てる IPアドレス（プライベート）を指定してください。

> VC管理者は、設定可能なIPアドレスの範囲を「VCP Manager Web UI」から確認することができます。

In [ ]:
# (例)
# vc_ipaddress = '172.30.2.100'

vc_ipaddress =

## SSH公開鍵認証の鍵ファイルの指定

起動したVCノードにsshでログインして操作を行うためには事前にSSH公開鍵認証の公開鍵を登録する必要があります。

VCノードに登録するSSHの公開鍵ファイルのパスを次のセルで指定してください。

> 事前に鍵ファイル`~/.ssh/id_rsa` を作成しておいてください。

In [ ]:
ssh_public_key_path = '~/.ssh/id_rsa.pub'

公開鍵に対応する秘密鍵のパスを次のセルで指定してください。

In [ ]:
ssh_private_key_path = '~/.ssh/id_rsa'

## UnitGroup名の指定

VCPの構成要素は以下のようになっています。

* VCノード
  - クラウドにおける計算資源(VM/BM)
  - 例えば Amazon EC2インスタンス, Microsoft Azure VM など
* Unit
  - 同質のVCノードにより構成されている要素
  - 同じUnitに属するVCノードはCPU,メモリ等の計算資源が全て同じ設定になっている
* UnitGroup
  - 複数のUnitにより構成されている要素
  - 使用目的、ライフサイクルなどに合わせて、複数のUnitをまとめて扱うための要素

このNotebookで構築するCoursewareHub環境では２つのUnitを作成することを想定しています。
* managerユニット
  - Docker Swarm の managerとなるVCノード
  - NFS用ディスクをアタッチする
* workerユニット
  - Docker Swarm の workerとなるVCノード

２つのUnitをまとめて扱うための UnitGroup の名前を、次のセルで指定してください。

In [ ]:
unit_group = 'CoursewareHub'

## パラメータの保存

この章で指定したパラメータの値をファイルに保存します。

後の手順でVCノードに対する操作を、構成管理ツールの[Ansible](https://www.ansible.com/)で行います。そこで、パラメータの保存形式は `Ansible` のフォーマットに従うことにします。Ansible では `group_vars/`というディレクトリに YAML フォーマットのファイルを配置すると、そのファイルに記録されている値を変数として利用することができます。このNotebookでは `group_vars/` にあるファイルを `group_vars ファイル`と呼ぶことにします。

次のセルを実行すると、この章で指定したパラメータが group_vars ファイルに保存されます。

> YAMLフォーマットでファイルに値を保存するために、事前に作成した Python のスクリプト `scripts/group.py` を利用しています。

In [ ]:
import os
%run scripts/group.py

# SSHの鍵ファイルが存在していることを確認します
if not (os.path.exists(os.path.expanduser(ssh_public_key_path)) and os.path.exists(os.path.expanduser(ssh_private_key_path))):
    raise

update_group_vars(
    unit_group,
    unit_group=unit_group,
    vc_provider=vc_provider,
    vc_flavor=vc_flavor,
    vc_nodes=vc_nodes,
    vc_disk_size=vc_disk_size,
    vc_ipaddress=vc_ipaddress,
    ssh_public_key_path=os.path.expanduser(ssh_public_key_path),
    ssh_private_key_path=os.path.expanduser(ssh_private_key_path),
)

上のセルがエラーになる場合は、この章で設定する必要があるパラメータが、まだ設定されていない可能性があります。この章のセルで実行していないことを確認してください。実行していないものがあった場合はそのセルを実行した後、エラーになったこの節のセルを再度実行して下さい。

group_vars ファイルの内容を表示して保存されたパラメータを確認します。

In [ ]:
!cat group_vars/{unit_group}.yml

# VCディスクに関するパラメータ

CoursewareHubに関するデータやNotebook、ユーザのホームディレクトリなどに利用するVCディスクに関するパラメータを指定します。

ディスクサイズ(GB)を指定してください。

> VCディスクを作成しないで全てのデータをルートボリュームに配置する場合は、このセクションをスキップするか、`vc_nfs_disk_size` を `0` に設定してください。

In [ ]:
# (例)
# vc_nfs_disk_size = 32

vc_nfs_disk_size =

VCディスクのためのUnitGroup名を指定してください。ここではVCノードのUnitGroup名から自動的に導き出した名前を設定します。

In [ ]:
disk_unit_group = unit_group + '_disk'
print(disk_unit_group)

この章で指定したパラメータの値をファイルに保存します。

In [ ]:
%run scripts/group.py

if 'vc_nfs_disk_size' in vars() and vc_nfs_disk_size > 0:
    update_group_vars(
        unit_group,
        disk_unit_group=disk_unit_group,
        vc_nfs_disk_size=vc_nfs_disk_size,
    )

group_vars ファイルの内容を表示して保存されたパラメータを確認します。

In [ ]:
!cat group_vars/{unit_group}.yml

# VCディスクの作成

NFS用のVCディスクを作成します。

![VCディスク](images/cw-001-02.png)

> 前のセクションをスキップしたか、または `vc_nfs_disk_size`の値として `0` を設定した場合は、このセクションを実行してもVCディスクは作成されません。

## UnitGroupの作成

VCディスクを管理するための UnitGroup を作成します。


UnitGroupを作成するまえに、現在のUnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

UnitGroupを作成します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(unit_group)

if 'disk_unit_group' in gvars and 'vc_nfs_disk_size' in gvars and gvars['vc_nfs_disk_size'] > 0:
    ug_disk = vcp.create_ugroup(gvars['disk_unit_group'], ugroup_type='storage')

UnitGroup作成後の一覧を表示させます。

In [ ]:
vcp.df_ugroups()

## VCディスクの作成

NFS用のVCディスクを作成します。

In [ ]:
if 'ug_disk' in vars() and gvars['vc_nfs_disk_size'] > 0:
    nfs_disk_spec = vcp.get_spec(gvars['vc_provider'] + '_disk', 'small')
    nfs_disk_spec.size = gvars['vc_nfs_disk_size']
    ug_disk.create_unit('nfs', nfs_disk_spec)

作成したVCディスクの一覧を表示します。

In [ ]:
if 'ug_disk' in vars():
    ug_disk.df_nodes()

# VCノードの起動


![VCノード](images/cw-001-03.png)

## manager用VCノードの spec を指定する

指定したパラメータをVCノードの `spec` に設定します。

In [ ]:
import sys
%run scripts/group.py
gvars = load_group_vars(unit_group)
spec_mgr = vcp.get_spec(gvars['vc_provider'], gvars['vc_flavor'])

# manager用の Baseコンテナイメージを指定する
spec_mgr.image = 'harbor.vcloud.nii.ac.jp/vcp/coursewarehub:manager'
spec_mgr.params_v = [
    '/sys/fs/cgroup:/sys/fs/cgroup:ro',
    '/lib/modules:/lib/modules:ro',
]

# manager用VCノードに割り当てるIPアドレスを指定する
spec_mgr.ip_addresses = [gvars['vc_ipaddress']]

# ルートボリュームサイズを指定する
if vc_provider == 'aws':
    spec_mgr.volume_size = gvars['vc_disk_size']
elif vc_provider == 'azure':
    spec_mgr.disk_size_gb = gvars['vc_disk_size']
else:
    print('This provider does not support the specification of the root volume size. Does not set the disk size.', file=sys.stderr)

# VCノードにsshでログインするための公開鍵を指定する
spec_mgr.set_ssh_pubkey(gvars['ssh_public_key_path'])

if 'ug_disk' in vars():
    spec_mgr.disks = ug_disk.find_nodes()
    if vc_provider == 'aws':
        spec_mgr.params_e.append("NFS_DEV=/dev/xvdf")
    elif vc_provider == 'azure':
        spec_mgr.params_e.append("NFS_DEV=/dev/sdc")
    else:
        print('This provider does not support the specification of the root volume size. Does not set the disk size.', file=sys.stderr)

`spec` の設定値を確認します。

In [ ]:
print(spec_mgr)

## worker用VCノードのspec を指定する

指定したパラメータをworker用VCノードの `spec` に設定します。

In [ ]:
spec_worker = vcp.get_spec(gvars['vc_provider'], gvars['vc_flavor'])

# worker用の Baseコンテナイメージを指定する
spec_worker.image = 'harbor.vcloud.nii.ac.jp/vcp/coursewarehub:worker'
spec_worker.params_v = [
    '/sys/fs/cgroup:/sys/fs/cgroup:ro',
    '/lib/modules:/lib/modules:ro',
]

# ルートボリュームサイズを指定する
if vc_provider == 'aws':
    spec_worker.volume_size = gvars['vc_disk_size']
elif vc_provider == 'azure':
    spec_worker.disk_size_gb = gvars['vc_disk_size']
else:
    print('This provider does not support the specification of the root volume size. Does not set the disk size.', file=sys.stderr)

# VCノードにsshでログインするための公開鍵を指定する
spec_worker.set_ssh_pubkey(gvars['ssh_public_key_path'])

# NFSサーバのIPアドレスを指定する
spec_worker.params_e.append("NFS_SERVER=" + gvars['vc_ipaddress'])

# ノード数を指定する
spec_worker.num_nodes = int(gvars['vc_nodes']) - 1

`spec` の設定値を確認します。

In [ ]:
print(spec_worker)

## VCノードの起動

UnitGroupを作成します。

In [ ]:
ugroup = vcp.create_ugroup(unit_group)

manager用VCノードを起動します。

In [ ]:
unit_mgr = ugroup.create_unit('manager', spec_mgr)

起動したVCノードの一覧を表示します。

In [ ]:
unit_mgr.df_nodes()

worker用VCノードを起動します。

In [ ]:
unit_worker = ugroup.create_unit('worker', spec_worker)

起動したVCノードの一覧を表示します。

In [ ]:
unit_worker.df_nodes()

# Ansibleの設定

VCノードをAnsibleで操作するための設定を行います。

まず、VCノードにSSHでログインできるようにするために `~/.ssh/known_hosts` の更新を行います。

> 何度かVCノードの起動を行うと、異なるホストが同じIPアドレスで起動するためにSSHのホストキーのチェックでエラーになる事があります。このような状況に対応するために、起動したVCノードのIPアドレスに対応するエントリを`known_hosts`ファイルから削除します。その後、`ssh-keyscan`コマンドを利用して起動したVCノードのホストキーを取得して `known_hosts`ファイルの内容を更新します。

In [ ]:
from time import sleep

def check_update_known_hosts(ipaddr):
    # VCノード起動直後だと sshd サービスが開始されておらずに known_hosts が更新されない場合がある
    # ssh-keyscan が値を取得できるまで何度かリトライする
    for x in range(10):
        out = ! echo $(ssh-keyscan {ipaddr} 2> /dev/null | wc -l)
        update_lines = int(out[0])
        if update_lines > 0:
            break
        sleep(1)
    else:
        raise RuntimeError("ERROR: timeout!")    

!mkdir -p -m 0700 ~/.ssh
for addr in ugroup.find_ip_addresses():
    !ssh-keygen -R {addr}
    check_update_known_hosts(addr)
    !ssh-keyscan -H {addr} >> ~/.ssh/known_hosts

起動したVCノードに対応するエントリを Ansible のインベントリに登録します。

> Ansibleで操作を行うためには、操作対象のホスト(IPアドレス)をインベントリに登録する必要があります。

In [ ]:
import yaml

hosts = {
    'all': {
        'children': {
            ugroup.name: {
                'children': {
                    unit_mgr.name: {
                        'hosts': dict([(x, {}) for x in unit_mgr.find_ip_addresses()]),
                    },
                    unit_worker.name: {
                        'hosts': dict([(x, {}) for x in unit_worker.find_ip_addresses()]),
                    }
                },
                'vars': {
                    'ansible_user': 'vcp',
                    'ansible_ssh_private_key_file': gvars['ssh_private_key_path'],
                }
            }
        }
    }
}

with open('hosts', 'w') as f:
    yaml.dump(hosts, f, default_flow_style=False)
    
!cat hosts

先程VCノードを登録したファイルをインベントリとして指定するためのAnsibleのコンフィギュレーションファイルを作成します。

> カレントディレクトリにコンフィギュレーションファイル(`ansible.cfg`)を作成すると、Ansibleを実行する際にその設定が適用されます。

In [ ]:
from pathlib import Path

inventory = Path('./hosts').resolve()
ansible_cfg = Path('./ansible.cfg').resolve()
with ansible_cfg.open(mode='w') as f:
    f.write(f'''
[defaults]
inventory = {inventory}
deprecation_warnings = False
force_valid_group_names = ignore
command_warnings = False
''')
ansible_cfg.parent.chmod(0o755)

!cat ./ansible.cfg

UnitGroupに属する全てのVCノードに対して Ansible で接続できることを確認します。

> ここでは、複数のVCノードをまとめて扱うためにAnsibleのグループを指定しています。グループ名は UnitGroup名と同じ値にしてあります。

In [ ]:
!ansible {ugroup.name} -m ping

VCノードにログインするユーザ名を `group_vars`に保存しておきます。

In [ ]:
%run scripts/group.py

update_group_vars(
    unit_group,
    default_user='vcp',
)

# Docker Swarmの設定

Docker Swarm の設定を行います。

![Docker Swarm](images/cw-001-04.png)

manager用VCノードで Docker Swarmの初期セットアップを行います。

In [ ]:
!ansible {unit_mgr.name} -a 'docker swarm init'

トークンの値を取得します。

In [ ]:
out = !ansible {unit_mgr.name} -a 'docker swarm join-token -q worker'
if out[0].find('CHANGED') >= 0 or out[0].find('SUCCESS') >= 0:
    swarm_token = out[1]
    print(swarm_token)
else:
    raise

workerのVCノードを追加します。

In [ ]:
manager_ip = gvars['vc_ipaddress']
!ansible {unit_worker.name} -a 'docker swarm join --token {swarm_token} {manager_ip}:2377'

Docker Swarmのノード一覧を表示します。

In [ ]:
!ansible {unit_mgr.name} -a 'docker node ls'